In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras import layers, Input, Model, optimizers

# Scratchwork

In [102]:
from skimage import io

imag = io.imread('data/RAW/1010-ExpBlock1_RAW.png', as_gray=True)
x = image.img_to_array(imag)
#n = preprocess_input(x)    # preprocess input for Inception v3

### (760, 598, 4)

In [104]:
# read, preprocess, and scale images to uniform dimension
imgsize = (760, 598)  
def extract_image(image_path):
    imag = io.imread(image_path, as_gray=True)
    x = image.img_to_array(imag)

    x = tf.keras.preprocessing.image.smart_resize(x, imgsize)
    
    n = preprocess_input(x)
    return n

In [6]:
# Use InceptionV3 model for feature extraction
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(None, None, 3),
    pooling='max',
)

In [107]:
# Garcia-Ceja paper feeds CNN images with dims 100 x 100 x 4 (width x height x channels)
# Labels are one hot encoded

inputs = Input(shape=(760, 598, 1))
x = layers.Conv2D(filters=16, kernel_size=3, strides=(1, 1), activation="relu")(inputs)
x = layers.Conv2D(filters=16, kernel_size=3, strides=(1, 1), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.25)(x)
x = layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), activation="relu")(x)
x = layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(512, activation="relu")(x) 
x = layers.Dropout(0.50)(x)
out = layers.Dense(1, activation=tf.nn.relu6)(x) 

# ---- Output layer ----
# Try a ReLU6 activation function -> f(x) = min(max(0,x),6). Restricted between 0 and 6, inclusive.
# Then scale the range [0,6] to our target [0, 15] range and round to nearest integer

model = Model(inputs=inputs, outputs=out)
model.compile(
  optimizer = 'adam',
  loss=tf.keras.losses.MeanSquaredError(),  
  metrics=['accuracy'],
)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 760, 598, 1)]     0         
                                                                 
 conv2d_112 (Conv2D)         (None, 758, 596, 16)      160       
                                                                 
 conv2d_113 (Conv2D)         (None, 756, 594, 16)      2320      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 378, 297, 16)     0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 378, 297, 16)      0         
                                                                 
 conv2d_114 (Conv2D)         (None, 376, 295, 32)      4640      
                                                           

## Load and preprocess data

In [105]:
import os
import random

# read data to np arrays
recurrence_plot_list = []
labels = []
for filename in os.listdir('data/RAW/'):
    t = extract_image('data/RAW/' + filename)
    
    # randomly generate y labels for now
    lab = random.randrange(15)
    
    recurrence_plot_list.append(t)
    labels.append(lab)

recurrence_plot_list = np.array(recurrence_plot_list)
labels = np.array(labels)

print(f"Number of samples: {recurrence_plot_list.shape[0]}")

Number of samples: 271


In [108]:
import sklearn
from sklearn.model_selection import train_test_split

# Split into train/test set
X_train, X_test, y_train, y_test = train_test_split(recurrence_plot_list, labels, train_size=0.75)

# Scale y train data from [0, 15] to [0, 6] for training the neural network
y_train = y_train/15 * 6

In [110]:
# fit model
info = model.fit(x=X_train, y=y_train, epochs=3, verbose=True)

Epoch 1/3
7/7 [==============================] - 159s 22s/step - loss: 8.8502 - accuracy: 0.0156
Epoch 2/3
7/7 [==============================] - 142s 20s/step - loss: 3.4713 - accuracy: 5.6384e-09
Epoch 3/3
7/7 [==============================] - 119s 17s/step - loss: 2.8689 - accuracy: 0.0000e+00
